<a href="https://colab.research.google.com/github/snehahs03/medical-image-analysis/blob/main/resnet1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/snehahs03/medical-image-analysis.git

Cloning into 'medical-image-analysis'...
remote: Enumerating objects: 3785, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 3785 (delta 9), reused 0 (delta 0), pack-reused 3763
Receiving objects: 100% (3785/3785), 336.97 MiB | 37.29 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Checking out files: 100% (3887/3887), done.


In [2]:
import json
import math
import os
import cv2
from PIL import Image
import numpy as np
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tensorflow.keras import backend as K
import gc
from functools import partial
from tqdm import tqdm
from sklearn import metrics
from collections import Counter
import json
import itertools
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D 
from keras.layers import Input, Lambda, Dense, Flatten
from keras.preprocessing import image
from glob import glob

In [3]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".jpg":
            img = read(PATH)
           
            img = cv2.resize(img, (RESIZE,RESIZE))
           
            IMG.append(np.array(img))
    return IMG

In [4]:
eczema_train = np.array(Dataset_loader("/content/medical-image-analysis/train/Eczema Photos", 224))
melanoma_train = np.array(Dataset_loader("/content/medical-image-analysis/train/Melanoma Skin Cancer Nevi and Moles",224))
psoriasis_train = np.array(Dataset_loader("/content/medical-image-analysis/train/Psoriasis pictures Lichen Planus and related diseases",224))
eczema_test = np.array(Dataset_loader("/content/medical-image-analysis/test/Eczema Photos",224))
melonoma_test = np.array(Dataset_loader("/content/medical-image-analysis/test/Melanoma Skin Cancer Nevi and Moles",224))
psoriasis_test = np.array(Dataset_loader("/content/medical-image-analysis/test/Psoriasis pictures Lichen Planus and related diseases", 224))


100%|██████████| 352/352 [00:02<00:00, 155.65it/s]


In [5]:
#labelling 
eczema_train_label = np.full(len(eczema_train),0)
melonoma_train_label = np.full(len(melanoma_train),1)
psoriasis_train_label = np.full(len(psoriasis_train),2)
eczema_test_label = np.full(len(eczema_test),0)
melonoma_test_label = np.full(len(melonoma_test),1)
psoriasis_test_label = np.full(len(psoriasis_test),2)

In [6]:
X_train = np.concatenate((eczema_train, melanoma_train, psoriasis_train), axis=0)
Y_train = np.concatenate((eczema_train_label, melonoma_train_label, psoriasis_train_label), axis=0)
X_test = np.concatenate((eczema_test, melonoma_test, psoriasis_test), axis=0)
Y_test = np.concatenate((eczema_test_label, melonoma_test_label, psoriasis_test_label), axis = 0)
print(Y_test.shape)
print(X_test.shape)

(777,)
(777, 224, 224, 3)


In [7]:
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]


In [8]:
Y_train = to_categorical(Y_train, num_classes= 3)
Y_test = to_categorical(Y_test, num_classes= 3)

#train and evaluation split
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, 
    test_size=0.3, 
    random_state=5
)

In [9]:
X_train.shape, Y_train.shape

((2172, 224, 224, 3), (2172, 3))

In [10]:
X_val.shape, Y_val.shape

((931, 224, 224, 3), (931, 3))

In [11]:
X_test.shape, Y_test.shape

((777, 224, 224, 3), (777, 3))

In [12]:
pre_trained_model = tensorflow.keras.applications.ResNet50(input_shape=(224,224, 3), include_top=False, weights="imagenet")

94773248/94765736 [==============================] - 1s 0us/step


In [13]:
for layer in pre_trained_model.layers:
    print(layer.name)
    if hasattr(layer, 'moving_mean') and hasattr(layer, 'moving_variance'):
        layer.trainable = True
        K.eval(K.update(layer.moving_mean, K.zeros_like(layer.moving_mean)))
        K.eval(K.update(layer.moving_variance, K.zeros_like(layer.moving_variance)))
    else:
        layer.trainable = False

print(len(pre_trained_model.layers))

input_1
conv1_pad
conv1_conv
conv1_bn
conv1_relu
pool1_pad
pool1_pool
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu
conv2_block1_0_conv
conv2_block1_3_conv
conv2_block1_0_bn
conv2_block1_3_bn
conv2_block1_add
conv2_block1_out
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_conv
conv2_block2_2_bn
conv2_block2_2_relu
conv2_block2_3_conv
conv2_block2_3_bn
conv2_block2_add
conv2_block2_out
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_conv
conv2_block3_2_bn
conv2_block3_2_relu
conv2_block3_3_conv
conv2_block3_3_bn
conv2_block3_add
conv2_block3_out
conv3_block1_1_conv
conv3_block1_1_bn
conv3_block1_1_relu
conv3_block1_2_conv
conv3_block1_2_bn
conv3_block1_2_relu
conv3_block1_0_conv
conv3_block1_3_conv
conv3_block1_0_bn
conv3_block1_3_bn
conv3_block1_add
conv3_block1_out
conv3_block2_1_conv
conv3_block2_1_bn
conv3_block2_1_relu
conv3_block2_2_conv
conv3_block2_2_bn
conv3_

In [14]:
last_layer = pre_trained_model.get_layer('conv5_block3_out')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 7, 7, 2048)


In [15]:
x = tensorflow.keras.layers.Flatten()(last_output)
x = tensorflow.keras.layers.Dense(512, activation='relu')(x)
x = tensorflow.keras.layers.Dropout(0.5)(x)
x = tensorflow.keras.layers.Dense(512, activation='relu')(x)
x = tensorflow.keras.layers.Dropout(0.5)(x)
x = tensorflow.keras.layers.Dense(3, activation='softmax')(x)

#Config and compile model

model1 = Model(pre_trained_model.input, x)
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model1.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

TRAINING 

In [16]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [17]:
batch_size = 64
epochs = 25
history0 = model1.fit_generator(train_datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, Y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
33/33 [==============================] - 67s 928ms/step - loss: 2.0488 - accuracy: 0.4768 - val_loss: 183330928.0000 - val_accuracy: 0.4799
Epoch 2/25
33/33 [==============================] - 28s 840ms/step - loss: 1.0594 - accuracy: 0.5607 - val_loss: 891.2230 - val_accuracy: 0.4509
Epoch 3/25
33/33 [==============================] - 28s 840ms/step - loss: 0.9382 - accuracy: 0.5745 - val_loss: 4.5475 - val_accuracy: 0.3683
Epoch 4/25
33/33 [==============================] - 28s 834ms/step - loss: 0.9029 - accuracy: 0.5769 - val_loss: 1.1117 - val_accuracy: 0.4509
Epoch 5/25
33/33 [==============================] - 28s 837ms/step - loss: 0.8679 - accuracy: 0.6044 - val_loss: 0.9485 - val_accuracy: 0.5737
Epoch 6/25
33/33 [==============================] - 28s 837ms/step - loss: 0.8545 - accuracy: 0.5911 - val_loss: 0.8648 - val_accuracy: 0.6272
Epoch 7/25
33/33 [==============================] - 28s 844ms/step - loss: 0.8338 - accuracy: 0.5949 - val_loss: 0.7808 - val_accura

In [18]:
model1.save("0thModel.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


**FINE TUNING - 1
training first 5 layers**

In [19]:
for layer in pre_trained_model.layers[0:6]:
    layer.trainable = True

In [20]:
model2 = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model2.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [21]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

In [22]:
batch_size = 64
epochs = 25
history1 = model2.fit_generator(train_datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, Y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
33/33 [==============================] - 33s 889ms/step - loss: 0.6673 - acc: 0.7315 - val_loss: 0.7215 - val_acc: 0.6897
Epoch 2/25
33/33 [==============================] - 28s 842ms/step - loss: 0.6563 - acc: 0.7168 - val_loss: 0.7079 - val_acc: 0.6853
Epoch 3/25
33/33 [==============================] - 28s 843ms/step - loss: 0.6067 - acc: 0.7306 - val_loss: 0.7397 - val_acc: 0.6763
Epoch 4/25
33/33 [==============================] - 28s 847ms/step - loss: 0.6670 - acc: 0.7116 - val_loss: 0.6566 - val_acc: 0.7188
Epoch 5/25
33/33 [==============================] - 28s 845ms/step - loss: 0.6555 - acc: 0.7130 - val_loss: 0.7331 - val_acc: 0.6719
Epoch 6/25
33/33 [==============================] - 28s 842ms/step - loss: 0.6197 - acc: 0.7339 - val_loss: 0.6296 - val_acc: 0.7143
Epoch 7/25
33/33 [==============================] - 28s 844ms/step - loss: 0.6052 - acc: 0.7491 - val_loss: 0.7050 - val_acc: 0.7031

Epoch 00007: ReduceLROnPlateau reducing learning rate to 4.999999873

In [23]:
model2.save("1stModel.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


**FINE TUNING- 2
10 LAYERS TRAINABLE**

In [24]:
for layer in pre_trained_model.layers[0:11]:
    layer.trainable = True


model3 = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model3.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])


learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

batch_size = 64
epochs = 25
history2 = model3.fit_generator(train_datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, Y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
33/33 [==============================] - 33s 873ms/step - loss: 0.5638 - acc: 0.7832 - val_loss: 0.7499 - val_acc: 0.7188
Epoch 2/25
33/33 [==============================] - 28s 843ms/step - loss: 0.5744 - acc: 0.7709 - val_loss: 0.6660 - val_acc: 0.7188
Epoch 3/25
33/33 [==============================] - 28s 852ms/step - loss: 0.5275 - acc: 0.7823 - val_loss: 0.7262 - val_acc: 0.7143
Epoch 4/25
33/33 [==============================] - 28s 840ms/step - loss: 0.5509 - acc: 0.7671 - val_loss: 0.6682 - val_acc: 0.7567
Epoch 5/25
33/33 [==============================] - 28s 844ms/step - loss: 0.5320 - acc: 0.7756 - val_loss: 0.7381 - val_acc: 0.7321
Epoch 6/25
33/33 [==============================] - 28s 842ms/step - loss: 0.5356 - acc: 0.7756 - val_loss: 0.6561 - val_acc: 0.7299
Epoch 7/25
33/33 [==============================] - 28s 843ms/step - loss: 0.5105 - acc: 0.7984 - val_loss: 0.6800 - val_acc: 0.6875

Epoch 00007: ReduceLROnPlateau reducing learning rate to 4.999999873

In [25]:
model2.save("2ndModel.h5")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [27]:
!pwd

/content


In [28]:
!git init

Initialized empty Git repository in /content/.git/


In [29]:
!git add 0thModel.h5

In [30]:
!git add 1stModel.h5

In [31]:
!git add 2ndModel.h5

In [32]:
!git status

On branch master

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)

	new file:   0thModel.h5
	new file:   1stModel.h5
	new file:   2ndModel.h5

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.config/
	medical-image-analysis/
	sample_data/



In [35]:
!git config --global user.email "hssneha03@gmail.com"
!git config --global user.name "snehahs03"

In [36]:
!git commit -m 'h5 files added of resnet50'

[master (root-commit) febe160] h5 files added of resnet50
 3 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 0thModel.h5
 create mode 100644 1stModel.h5
 create mode 100644 2ndModel.h5


In [47]:
!git remote add origin https://github.com/snehahs03/medical-image-analysis.git

fatal: remote origin already exists.


In [48]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address


**FINE TUNING - 3**

In [26]:
for layer in pre_trained_model.layers[0:11]:
    layer.trainable = True


model4 = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model4.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])


learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

batch_size = 64
epochs = 25
history3 = model4.fit_generator(train_datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, Y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
33/33 [==============================] - 33s 871ms/step - loss: 0.4964 - acc: 0.8164 - val_loss: 0.7038 - val_acc: 0.7545
Epoch 2/25
33/33 [==============================] - 28s 841ms/step - loss: 0.4637 - acc: 0.8027 - val_loss: 0.6795 - val_acc: 0.7299
Epoch 3/25
23/33 [===================>..........] - ETA: 7s - loss: 0.4702 - acc: 0.8106

KeyboardInterrupt: ignored

In [ ]:
model3.save("3rdModel.h5")

**FINE TUNING - 4**

In [ ]:
y = tensorflow.keras.layers.Flatten()(last_output)
y = tensorflow.keras.layers.Dense(512, activation='relu')(y)
y = tensorflow.keras.layers.Dropout(0.5)(y)
y = tensorflow.keras.layers.Dense(3, activation='softmax')(y)

In [ ]:
for layer in pre_trained_model.layers[11:25]:
    layer.trainable = True


model5 = Model(pre_trained_model.input, y)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model4.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])


learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

batch_size = 64
epochs = 25
history4 = model4.fit_generator(train_datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, Y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

In [ ]:
model3.save("4thModel.h5")

In [ ]:
# Retrieve a list of accuracy results on training and test data
# sets for each training epoch
acc0 = np.array(history0.history['val_accuracy'])
acc1 = np.array(history1.history['val_acc'])
acc2 = np.array(history2.history['val_acc'])
acc3 = np.array(history3.history['val_acc'])
acc4 = np.array(history4.history['val_acc'])

# Retrieve a list of list results on training and test data
# sets for each training epoch
loss0 = history0.history['val_loss']
loss1 = history1.history['val_loss']
loss2 = history2.history['val_loss']
loss3 = history2.history['val_loss']
loss4 = history3.history['val_loss']


# Get number of epochs
epochs = range(len(acc0))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc0, label = "training")
plt.plot(epochs, acc1, label = "fine_tuning1")
plt.plot(epochs, acc2, label = "fine_tuning2")
plt.plot(epochs, acc3, label = "fine_tuning3")
plt.plot(epochs, acc4, label = "fine_tuning3")
plt.legend(loc="upper left")
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss0, label = "training")
plt.plot(epochs, loss1, label = "fine_tuning1")
plt.plot(epochs, loss2, label = "fine_tuning2")
plt.plot(epochs, loss3, label = "fine_tuning3")
plt.plot(epochs, loss4, label = "fine_tuning4")
plt.legend(loc="upper right")
plt.title('Training and validation loss')

In [ ]:
loss_test, acc_test = model.evaluate(X_test, Y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

In [ ]:
model.save("InceptionResNet.h5")

In [ ]:
# Retrieve a list of accuracy results on training and test data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and test data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, label = "training")
plt.plot(epochs, val_acc, label = "validation")
plt.legend(loc="upper left")
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss, label = "training")
plt.plot(epochs, val_loss, label = "validation")
plt.legend(loc="upper right")
plt.title('Training and validation loss')

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred,axis = 1) 
# Convert validation observations to one hot vectors
y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(3))